# AV: can we put a man in the middle there so the workflow to route to one node or another based on what LLM understands from the user input?

In [60]:
with open("./langgraph-thinking.md", "r") as f: 
    input_text = f.read()

input_text[:100]

'> ## Documentation Index\n> Fetch the complete documentation index at: https://docs.langchain.com/llm'

In [61]:
from langgraph.graph import StateGraph, MessagesState
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5-mini", model_provider="openai")

llm.invoke('hi')

AIMessage(content='Hi — how can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 7, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D7mPzm0iZPHUXfpVBjdpi03Jju0TB', 'finish_reason': 'stop', 'logprobs': None}, id='run-1313e4e7-a093-45bc-9eaa-33dd3f17d8d8-0', usage_metadata={'input_tokens': 7, 'output_tokens': 82, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [62]:
from pydantic import BaseModel, Field


class RouterOutput(BaseModel):
    route: str = Field(description="The route to take based on the user's input, options: summarize, extract")

llm_router = llm.with_structured_output(RouterOutput)

llm_router.invoke('extract info from this paper')

RouterOutput(route='extract')

In [65]:
def user_input_router_node(state: CustomState):
    """Node: gets user input, runs LLM router, and stores the chosen route in state.
    Nodes must return a state update (dict), not the next node name.
    No need to pass through input_text -- StateGraph preserves fields automatically."""
    user_input = input("What do you want to do with the doc? ")
    llm_output = llm_router.invoke(user_input)
    if llm_output.route == 'summarize':
        next_route = "summarize_node"
    elif llm_output.route == 'extract':
        next_route = "extract_node"
    else:
        raise ValueError(f"Invalid route: {llm_output.route}")
    return {"next_route": next_route}

In [71]:
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage

llm = init_chat_model("gpt-5-mini",
                                 model_provider="openai",
                                 )


def summarize_node(state: CustomState):
    # input_text is a str from f.read(); wrap it in a HumanMessage for the LLM
    raw_input_text = state["input_text"]
    summary = llm.invoke(
        [
            SystemMessage(content="You summarize text in bullet points covering the main information."),
            HumanMessage(content=raw_input_text),
        ]
    )
    return {"messages": [summary]}

def extract_node(state: CustomState):
    """
    Node: extracts structured information from the document using a custom Pydantic model and LangGraph.
    Returns: a state update with extracted info as a list of dicts in 'messages'.
    """

    # Define your custom Pydantic output schema for extraction
    class ExtractedInfo(BaseModel):
        title: str = Field(description="Title of the document")
        authors: list[str] = Field(description="List of authors")
        publication_year: int = Field(description="Year of publication")
        main_findings: str = Field(description="Main findings of the paper")

    # Wrap the LLM with the structured output schema
    llm_extractor = llm.with_structured_output(ExtractedInfo)

    raw_input_text = state["input_text"]
    # Call extraction LLM
    extracted = llm_extractor.invoke(raw_input_text)
    extracted_info_str = f"Title: {extracted.title}\nAuthors: {', '.join(extracted.authors)}\nPublication Year: {extracted.publication_year}\nMain Findings: {extracted.main_findings}"
    # You may choose to add to messages or a new field
    return {"messages": [extracted_info_str]}
    

In [67]:
def route_after_user_input(state: dict):
    """Router: reads the route from state and returns the next node name.
    Used by add_conditional_edges; only routing functions return node names."""
    return state["next_route"]

In [72]:
from IPython.display import Markdown
from langgraph.graph import START, END

class CustomState(MessagesState):
    input_text: str

builder = StateGraph(CustomState)

builder.add_node("user_input", user_input_router_node)
builder.add_node("summarize_node", summarize_node)
builder.add_node("extract_node", extract_node)
builder.add_conditional_edges("user_input", route_after_user_input)
builder.add_edge(START, "user_input")
# Router must be a function that takes state and returns next node name (not the node itself)
builder.add_edge("summarize_node", END)
builder.add_edge("extract_node", END)
graph = builder.compile()

output = graph.invoke({"input_text": input_text})

# final output summary
# Markdown(output["messages"][-1].content)

In [73]:
Markdown(output["messages"][-1].content)

Title: Thinking in LangGraph
Authors: LangChain Documentation
Publication Year: 2024
Main Findings: This guide explains how to design and build durable, inspectable AI agents with LangGraph by decomposing workflows into discrete nodes, defining shared raw state, and wiring nodes with explicit routing via Command objects. Key recommendations: break processes into small nodes (LLM, data, action, and human-input types), store only raw data in state and format prompts on-demand, handle errors with appropriate strategies (retry for transient, interrupt for user-fixable, bubble up unexpected), use interrupt() for human-in-the-loop pauses with persistent checkpoints, and add retry policies and checkpoints to nodes that call external services. The walkthrough implements a customer-support-email agent (read email, classify intent, search docs, bug tracking, draft reply, human review, send reply) with example node code, state schema, and graph compilation guidance.

In [74]:
# output = graph.invoke({"input_text": input_text})

In [70]:
Markdown(output["messages"][-1].content)

- Purpose: walkthrough for designing a customer-support email agent with LangGraph using a node/state-based workflow.

- High-level workflow (nodes):
  - Read Email
  - Classify Intent (intent, urgency, topic, summary)
  - Doc Search (knowledge base)
  - Bug Track (create/update issues)
  - Draft Reply
  - Human Review (interrupt/resume)
  - Send Reply

- Example scenarios the agent should handle:
  - Password reset question, bug reports, urgent billing, feature requests, complex technical issues.

- Design approach (five steps):
  1. Map the process into discrete nodes and possible transitions.
  2. Identify each node’s role: LLM steps, data steps, action steps, user-input steps.
  3. Design shared state (store raw data only; format prompts on-demand).
  4. Implement nodes as functions that take state and return updates (use Command to route).
  5. Wire nodes into a StateGraph, compile, and run (use a checkpointer to persist state for interrupts).

- State design guidance:
  - Store raw, minimal data needed across steps (do not store formatted prompts).
  - Example state structure (EmailAgentState):
    - email_content, sender_email, email_id
    - classification (EmailClassification dict or None)
    - search_results (list[str] or None)
    - customer_history (dict or None)
    - draft_response (str or None)
    - messages (list[str] or None)
  - EmailClassification fields: intent (question, bug, billing, feature, complex), urgency (low/medium/high/critical), topic, summary.

- Node implementation patterns:
  - Nodes are small functions: read_email, classify_intent, search_documentation, bug_tracking, draft_response, human_review, send_reply.
  - Use LLM steps for classification and drafting; use structured output when possible.
  - Nodes decide routing and return Command(update=..., goto="next_node").
  - Format prompts at call-time using raw state.

- classify_intent specifics:
  - Use structured LLM output (EmailClassification).
  - Route based on intent/urgency: e.g., billing or critical → human_review; question/feature → search_documentation; bug → bug_tracking.

- search_documentation and bug_tracking:
  - Store raw search results or ticket IDs in state.
  - Add retry logic for transient search errors and handle recoverable errors by storing error info.

- draft_response:
  - Build prompt from classification, search results, and customer history formatted on-demand.
  - Generate response with LLM and decide if human review is needed (high/critical urgency or complex intent).

- human_review:
  - Use interrupt() to pause for human input; interrupt must be the first action in the node (code before it will re-run on resume).
  - Resume with human decision: approved → send_reply; rejected → END (human handles).

- send_reply:
  - Perform final action (call email service); unexpected exceptions should bubble up.

- Error handling strategies:
  - Transient errors: system retries (exponential backoff).
  - LLM-recoverable errors: capture in state and loop back so LLM can adjust.
  - User-fixable errors: pause with interrupt() and request input.
  - Unexpected errors: let them bubble up for developer debugging.

- Durable execution and persistence:
  - Use a checkpointer (e.g., MemorySaver) for interrupts and long-running flows.
  - Provide thread_id in config to group conversation state.
  - LangGraph checkpoints at node boundaries (async durability by default); durability modes can be tuned (exit/sync).

- Graph wiring:
  - Minimal explicit edges needed because nodes handle routing with Command objects.
  - Example edges: START→read_email, read_email→classify_intent, send_reply→END.

- Testing / run flow:
  - Invoke the compiled app with initial state and config (thread_id).
  - App pauses at interrupt and returns interrupt token.
  - Resume by invoking with a Command containing resume payload (approved/edited_response).

- Design trade-offs:
  - Smaller nodes = finer checkpoints, better observability, easier retries/testing, clearer failure isolation.
  - Larger nodes may be simpler but risk redoing more work on failure and reduce observability.

- Key principles & tips:
  - Break tasks into single-purpose nodes.
  - Keep state raw and format when needed.
  - Make routing explicit via Command return types.
  - Treat human-in-the-loop as first-class via interrupt/resume.

- Next steps / extensions:
  - Human-in-the-loop patterns (batch approvals, tool approvals).
  - Subgraphs for complex tasks.
  - Streaming for real-time progress.
  - Observability (LangSmith) and logging/monitoring.
  - Integrate more tools (web search, databases, APIs).
  - Implement retry/caching strategies per-node as needed.

- Summary insight:
  - LangGraph encourages decomposition into nodes, shared raw state, explicit routing via commands, durable execution with checkpoints, and clear handling of errors and human interventions.

# can i have multiple workflows, and the agent to decide which workflow to start? what happens if the human in the loop (got the nomenclature right now) asks a question that is not in context and tries to jump to the other workflow midway? or asks something random? how we can deal with that?